<div class="alert alert-block alert-success">
    <h1 align="center">Machine Learning in Python</h1>
    <h3 align="center">mohammad Malekpour</h3>
</div>

<center><img src = "https://www.ukri.org/wp-content/uploads/2020/10/UKRI-221020-EpidemiologyWhatItIs-GettyImages-735x490.jpg" width=50%>

[image source](https://www.ukri.org/our-work/tackling-the-impact-of-covid-19/understanding-coronavirus-covid-19-and-epidemics/epidemiology-what-it-is-and-how-its-mapping-the-spread-of-covid-19/)

# <h1 style='background:#26A2AB; border:0; color:black'><center>Table of contents</center></h1> 

* [Introduction](#0)
* [EDA](#1)
    * [What vaccines are used and in which countries?](#1.1)
        * [Johnson&Johnson](#1.1.1)
        * [Oxford/AstraZeneca](#1.1.2)
        * [Pfizer/BioNTech](#1.1.3)
        * [Sinopharm/Beijing](#1.1.4)
        * [Sinovac](#1.1.5)
        * [Sputnik V](#1.1.6)
        * [Moderna](#1.1.7)
        * [CanSino](#1.1.8)
        * [Covaxin](#1.1.9)
        * [Sinopharm/Wuhan](#1.1.10)
        * [ZF2001](#1.1.12)
        * [Abdala](#1.1.13)
        * [Soberana 02](#1.1.14)
        * [COVIran Barekat](#1.1.15)
        * [QazVac](#1.1.16)
        * [Sputnik Light](#1.1.17)
        * [EpiVacCorona](#1.1.18)
        * [Medigen](#1.1.19)
* [Data Preprocessing](#2)
    * [Removing Duplicates Datas ](#2.1)
    * [Lost DATA  ](#2.2)
* [Visualization](#3)
    * [Divide the vaccine by the number of countries used](#3.1)
    * [Total vaccinations per country, grouped by vaccine scheme](#3.2)
    * [People fully vaccinated per hundred](#3.3)
    * [Total vaccinations per months in 2021](#3.4)
* [Prepare Data for Machine learning](#4)
    * [Define input and output](#4.1)
    * [Normalize data](#4.2)
* [Train your model](#5)
    * [Random Forest](#5.1)
    * [Use GridSearch to select Best Hyperparameters](#5.2)
    * [Train Model](#5.3)
* [Test the model and show the metrics](#6)


# <a id="0"></a><h1 style='background:#26A2AB; border:0; color:black'><center>Introduction</center></h1> 

This Notebook would not have been possible without the fantastic dataset provided by [@Gabriel Preda](https://www.kaggle.com/gpreda).


The data contains the following information:  

* **Country** - this is the country for which the vaccination information is provided;     
* **Country ISO Code** - ISO code for the country;   
* **Date**- date for the data entry; for some of the dates we have only the daily vaccinations, for others, only the (cumulative) total;   
* **Total number of vaccin ations** - this is the absolute number of total immunizations in the country;  
* **Total number of people vaccinated** - a person, depending on the immunization scheme, will receive one or more (typically 2) vaccines; at a certain moment, the number of vaccination might be larger than the number of people;  
* **Total number of people fully vaccinated** - this is the number of people that received the entire set of immunization according to the immunization scheme (typically 2); at a certain moment in time, there might be a certain number of people that received one vaccine and another number (smaller) of people that received all vaccines in the scheme;  
* **Daily vaccinations (raw)** - for a certain data entry, the number of vaccination for that date/country;  
* **Daily vaccinations** - for a certain data entry, the number of vaccination for that date/country;  
* **Total vaccinations per hundred** - ratio (in percent) between vaccination number and total population up to the date in the country;  
* **Total number of people vaccinated per hundred** - ratio (in percent) between population immunized and total population up to the date in the country;  
* **Total number of people fully vaccinated per hundred** - ratio (in percent) between population fully immunized and total population up to the date in the country;   
* **Number of vaccinations per day** - number of daily vaccination for that day and country;   
* **Daily vaccinations per million** -  ratio (in ppm) between vaccination number and total population for the current date in the country;    
* **Vaccines used in the country** - total number of vaccines used in the country (up to date);    
* **Source name** - source of the information (national authority, international organization, local organization etc.);   
* **Source website** - website of the source of information;

The last data update is on the 3 December 2021. You can use the following link to view up-to-date [data](https://www.kaggle.com/gpreda/covid-world-vaccination-progress)

## <a id="0.1">Importing the libraries

In [2]:
# Ignore  the warnings
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

# EDA & Data Preprocessing & Visualization
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
import sklearn
from sklearn.preprocessing import StandardScaler
from pandas_profiling import ProfileReport
import plotly.express as px
import scipy
import plotly.graph_objs as go
from plotly.offline import iplot

# sets matplotlib to inline and displays graphs below the corressponding cell.
%matplotlib inline  
style.use('fivethirtyeight')
sns.set(style='whitegrid',color_codes=True)


#import the necessary regression  algoritme:

from sklearn.ensemble import RandomForestRegressor   # Random ForestRegressor
import xgboost as xgb                                # XGBoost


#model selection
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV

#evaluation metrics
from sklearn.metrics import r2_score

# save Model
import joblib

## <a id="0.2">Load Data

In [3]:
df = pd.read_csv('country_vaccinations.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'country_vaccinations.csv'

# <a id="1"></a><h1 style='background:#26A2AB; border:0; color:black'><center>EDA</center></h1> 

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.head(3)

Generate a profile report from a Dataset stored as a pandas DataFrame, output its content as an HTML report in a Jupyter notebook.

In [ ]:
prifile = ProfileReport(df)
prifile.to_notebook_iframe()

In [ ]:
print(df.country.unique().tolist())

## <a id="1.1">What vaccines are used and in which countries?

In [ ]:
list_of_Vaccines = []
for sublist in list(df.vaccines.map(lambda i : i.split(','))):
    for i in sublist:
        i = i.strip()   # remove leading and trailing whitespaces
        if i not in list_of_Vaccines:
            list_of_Vaccines.append(i)

list_of_Vaccines

_Vaccines typically require years of research and testing before reaching the clinic, but in 2020, scientists embarked on a race to produce safe and effective coronavirus vaccines in record time. Researchers are currently testing 110 vaccines in clinical trials on humans, and 43 have reached the final stages of testing. More than 75 preclinical vaccines are under active investigation in animals.The Vaccines that are used all around the world now: (03.12.2021)_
<br>
<br>

|VACCINE COMPANY| VACCINE NAME| EFFICACY| DOSE | TYPE | STORAGE|
|--- | --- |--- | --- | --- | ---|
|Johnson&Johnson	| Ad26.COV2.S	|72% in United States, 68% in Brazil and 64% in South Africa |	1 dose | Muscle injection | Up to two years frozen at –4° F (–20° C), and up to 6 months refrigerated at 36–46° F (2–8° C)|
|Oxford/AstraZeneca	|AZD1222	| 74% against symptomatic Covid19; 100% against severe or critical Covid-19. |	2 doses | Muscle injection | Stable in refrigerator for at least 6 months|
|Pfizer/BioNTech	|Comirnaty	|91% |	2 doses, 3 weeks apart | Muscle injection | Freezer storage only at –13°F to 5°F (–25°C to –15°C)|
|Sinopharm/Beijing	|BBIBP-CorV	|78.1% |	2 doses, 3 weeks apart | Muscle injection | |
|Sinovac         	|CoronaVac 	|50.65% in Brazil trial, 83.5% in Turkey trial |	2 doses, 3 weeks apart | Muscle injection |Refrigerated  |
|The Gamaleya Research Institute|Sputnik V |91.6% |	2 doses, 3 weeks apart | Muscle injection | Freezer storage. Developing an alternative formulation that can be refrigerated |
|Moderna	        |mRNA-1273 	| Preventing Covid-19 illness: 93.2%. Preventing severe disease: 98.2% | 2 doses, 4 weeks apart | Muscle injection |  30 days with refrigeration, 6 months at –4°F (–20°C) |
|CanSino        	| Convidecia | 63.7% | Single dose | Muscle injection | Refrigerated |
|Bharat Biotech     | Covaxin    | 77.8% |  2 doses, 4 weeks apart |  | At least a week at room temperature |
|Sinopharm/Wuhan    | BBIBP-CorV | 78.1% |  2 doses, 3 weeks apart | Muscle injection |  |
|Anhui Zhifei Longcom | ZF2001 | 81.76% | 3 doses, 4 weeks apart | Muscle injection |  |
|Center for Genetic Engineering and Biotechnology of Cuba    | Abdala | 92.28% |   |   |  |
|Cuba’s Finlay Vaccine Institute         | Soberana 02, or PastoCoVac | 71% with two doses, 92.4 % with Soberana Plus booster |   |   | Refrigerated |
|Shafa Pharmed Pars    | COVIran Barekat  |  |  |   |  |
|The central Asian nation of Kazakhstan             | QazVac |  |   |   |  |
|Sputnik Light      | Gamaleya researchers started a trial in which they gave people only the first dose of Ad26 adenoviruses |
| Vector Institute      | EpiVacCorona | Unknown |  2 doses, 3 weeks apart | Muscle injection | Stable in refrigerator for up to two years |
|Taiwan-based vaccine maker Medigen           | Medigen |  |  |  |  |


[source: The New York Times](https://www.nytimes.com/interactive/2020/science/coronavirus-vaccine-tracker.html)
<br>


#### <a id="1.1.1">Johnson&Johnson

In [ ]:
countries_use_Johnson = df[df.vaccines.str.contains('Johnson&Johnson')].sort_values(by="total_vaccinations", ascending= False)['country'].unique()
print(countries_use_Johnson.tolist())

#### <a id="1.1.2">Oxford/AstraZeneca

In [ ]:
countries_use_AstraZeneca = df[df.vaccines.str.contains('Oxford/AstraZeneca')].sort_values(by="total_vaccinations", ascending= False)['country'].unique()
print(countries_use_AstraZeneca.tolist())

#### <a id="1.1.3">Pfizer/BioNTech

In [ ]:
countries_use_Pfizer = df[df.vaccines.str.contains('Pfizer/BioNTech')].sort_values(by="total_vaccinations", ascending= False)['country'].unique()
print(countries_use_Pfizer.tolist())

#### <a id="1.1.4">Sinopharm/Beijing

In [ ]:
countries_use_Sinopharm_Beijing = df[df.vaccines.str.contains('Sinopharm/Beijing')].sort_values(by="total_vaccinations", ascending= False)['country'].unique()
print(countries_use_Sinopharm_Beijing.tolist())

#### <a id="1.1.5">Sinovac 

In [ ]:
countries_use_Sinovac = df[df.vaccines.str.contains('Sinovac')].sort_values(by="total_vaccinations", ascending= False)['country'].unique()
print(countries_use_Sinovac.tolist())

#### <a id="1.1.6">Sputnik V

In [ ]:
countries_use_Sputnik_V = df[df.vaccines.str.contains('Sputnik V')].sort_values(by="total_vaccinations", ascending= False)['country'].unique()
print(countries_use_Sputnik_V.tolist())

#### <a id="1.1.7">Moderna

In [ ]:
countries_use_Moderna = df[df.vaccines.str.contains('Moderna')].sort_values(by="total_vaccinations", ascending= False)['country'].unique()
print(countries_use_Moderna.tolist())

#### <a id="1.1.8">CanSino

In [ ]:
countries_use_CanSino = df[df.vaccines.str.contains('CanSino')].sort_values(by="total_vaccinations", ascending= False)['country'].unique()
print(countries_use_CanSino.tolist())

#### <a id="1.1.9">Covaxin

In [ ]:
countries_use_Covaxin = df[df.vaccines.str.contains('Covaxin')].sort_values(by="total_vaccinations", ascending= False)['country'].unique()
print(countries_use_Covaxin.tolist())

#### <a id="1.1.10">Sinopharm/Wuhan

In [ ]:
countries_use_Sinopharm_Wuhan = df[df.vaccines.str.contains('Sinopharm/Wuhan')].sort_values(by="total_vaccinations", ascending= False)['country'].unique()
print(countries_use_Sinopharm_Wuhan.tolist())

#### <a id="1.1.11">ZF2001

In [ ]:
countries_use_ZF2001 = df[df.vaccines.str.contains('ZF2001')].sort_values(by="total_vaccinations", ascending= False)['country'].unique()
print(countries_use_ZF2001.tolist())

#### <a id="1.1.12">Abdala

In [ ]:
countries_use_Abdala = df[df.vaccines.str.contains('Abdala')].sort_values(by="total_vaccinations", ascending= False)['country'].unique()
print(countries_use_Abdala.tolist())

#### <a id="1.1.13">Soberana 02

In [ ]:
countries_use_Soberana02 = df[df.vaccines.str.contains('Soberana02')].sort_values(by="total_vaccinations", ascending= False)['country'].unique()
print(countries_use_Soberana02.tolist())

#### <a id="1.1.14">COVIran Barekat

In [ ]:
countries_use_COVIran_Barekat = df[df.vaccines.str.contains('COVIran Barekat')].sort_values(by="total_vaccinations", ascending= False)['country'].unique()
print(countries_use_COVIran_Barekat.tolist())

#### <a id="1.1.15">QazVac

In [ ]:
countries_use_QazVac = df[df.vaccines.str.contains('QazVac')].sort_values(by="total_vaccinations", ascending= False)['country'].unique()
print(countries_use_QazVac.tolist())

#### <a id="1.1.16">Sputnik Light

In [ ]:
countries_use_Sputnik_Light = df[df.vaccines.str.contains('Sputnik Light')].sort_values(by="total_vaccinations", ascending= False)['country'].unique()
print(countries_use_Sputnik_Light.tolist())

#### <a id="1.1.17">EpiVacCorona

In [ ]:
countries_use_EpiVacCorona = df[df.vaccines.str.contains('EpiVacCorona')].sort_values(by="total_vaccinations", ascending= False)['country'].unique()
print(countries_use_EpiVacCorona.tolist())

#### <a id="1.1.18">Medigen

In [ ]:
countries_use_Medigen = df[df.vaccines.str.contains('Medigen')].sort_values(by="total_vaccinations", ascending= False)['country'].unique()
print(countries_use_Medigen.tolist())

# <a id="2"></a><h1 style='background:#26A2AB; border:0; color:black'><center>Data Preprocessing</center></h1> 

In [ ]:
df.drop(['source_name' , 'source_website'] , axis=1 , inplace=True)

## <a id="2.1">Removing Duplicates Datas 

In [ ]:
df.drop_duplicates(inplace = True)

In [ ]:
df.isna().sum()

## <a id="2.2">Lost DATA 

To preprocess lost data, it is first necessary to know the relationships between the data

In [ ]:
cor_mat= df.corr()

corr = cor_mat.corr()
# size
sns.set(rc={'figure.figsize':(20,10)})
# mask

mask = np.array(cor_mat)
mask[np.tril_indices_from(mask)] = False
sns.heatmap(corr,mask=mask, cmap="RdGy", annot=True)

As can be seen, the values of a number of columns are completely related. If the data are similar, the related data can be used to fill in the NAN values.
<br>
*Mann-Whithey U test* is the method used for check the hypothesis that these columns distributions are the same.

##### total_vaccinations, people_vaccinated and people_fully_vaccinated
As can be seen, the values in columns total_vaccinations, people_vaccinated and people_fully_vaccinated are completely related.


Let's check Mann-Whithey U test:

In [ ]:
scipy.stats.mannwhitneyu(df.total_vaccinations, df.people_vaccinated , alternative='two-sided')

The pvalue is less than 0.05 so Null Hypothesis is **Rejected** (Significant difference between two samples)

In [ ]:
scipy.stats.mannwhitneyu(df.total_vaccinations, df.people_fully_vaccinated , alternative='two-sided')

The pvalue is  much than 0.05 which means Null Hypothesis  **can't Rejected** (No significant difference between two samples)
<br><br>
But wait, it's important to first drop the raws that have't data in this three culomns then check Mann-Whithey U test.
drop total_vaccinations missing data,people_vaccinated and people_fully_vaccinated as without this values any raw doesn't make much sense.

In [ ]:
df.drop(df[ df.total_vaccinations.isna() & (df.people_vaccinated.isna() | df.people_fully_vaccinated.isna()) ].index , inplace=True)

In [ ]:
scipy.stats.mannwhitneyu(df.total_vaccinations, df.people_fully_vaccinated  , alternative='two-sided')

As you can see the pvalue is become less than 0.05 so Null Hypothesis is **Rejected**

In [ ]:
df.isna().sum()

To fill the NAN values two methoded is used:
<br>
* use the  interpolation values  of the previous and next raws are used
* use the last value
<br>
the first method used for the columns with few NAN values and the second method used for the columns with many NAN values.

In [ ]:
df.interpolate(method='linear', limit=30 ,limit_direction='both' , inplace=True )

In [ ]:
df.isna().sum()

In [ ]:
df.fillna(method="ffill" , inplace=True)

In [ ]:
df.isna().sum()

# <a id="3"></a><h1 style='background:#26A2AB; border:0; color:black'><center>Visualization</center></h1> 

## <a id="3.1">Divide the vaccine by the number of countries used

In [ ]:
size  = np.array([len(countries_use_Johnson) , len(countries_use_AstraZeneca) , len(countries_use_Pfizer) , len(countries_use_Sinopharm_Beijing)
       , len(countries_use_Sinovac) , len(countries_use_Sputnik_V) , len(countries_use_Moderna) , len(countries_use_CanSino)
       , len(countries_use_Covaxin) , len(countries_use_Sinopharm_Wuhan) , len(countries_use_ZF2001) , len(countries_use_Abdala)
       , len(countries_use_Soberana02) , len(countries_use_COVIran_Barekat) , len(countries_use_QazVac) , len(countries_use_Sputnik_Light)
       , len(countries_use_EpiVacCorona) , len(countries_use_Medigen)])/223*100

percent_Vac_num =  pd.DataFrame({'Vaccines':list_of_Vaccines , 'per':size , 'countries':[ '\n'.join(countries_use_Johnson) , ','.join(countries_use_AstraZeneca) 
       , ','.join(countries_use_Pfizer) , ','.join(countries_use_Sinopharm_Beijing) , ','.join(countries_use_Sinovac) , ','.join(countries_use_Sputnik_V) 
       , ','.join(countries_use_Moderna) , ','.join(countries_use_CanSino) , ','.join(countries_use_Covaxin) , ','.join(countries_use_Sinopharm_Wuhan) 
       , ','.join(countries_use_ZF2001) , ','.join(countries_use_Abdala) , ','.join(countries_use_Soberana02) , ','.join(countries_use_COVIran_Barekat) 
       , ','.join(countries_use_QazVac) , ','.join(countries_use_Sputnik_Light) , ','.join(countries_use_EpiVacCorona) , ','.join(countries_use_Medigen)  ]})
percent_Vac_num.sort_values(by='per' , ascending= False , inplace=True , ignore_index=True )

In [ ]:
fig = px.treemap(percent_Vac_num, 
                path = [px.Constant("all") ,'Vaccines' ], 
                values = 'per',
                title="Divide the vaccine by the number of countries used"
                )
fig.data[0].texttemplate = "%{label}<br>%{value:.2f}"
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig.show()

## <a id="3.2">Total vaccinations per country, grouped by vaccine scheme

In [ ]:
fig = px.treemap(df, path = ['vaccines', 'country'], values = 'total_vaccinations',
                title="Total vaccinations per country, grouped by vaccine scheme")

fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig.show()

## <a id="3.3">People fully vaccinated per hundred

In [ ]:
trace = go.Choropleth(
            locations = df['country'],
            locationmode='country names',
            z = df['people_fully_vaccinated_per_hundred'],
            text = df['country'],
            autocolorscale =False,
            reversescale = True,
            colorscale = 'cividis',
            marker = dict(
                line = dict(
                    color = 'rgb(0,0,0)',
                    width = 0.5)
            ),
            colorbar = dict(
                title = 'People fully vaccinated per hundred',
                tickprefix = '')
        )

data = [trace]
layout = go.Layout(
    title = 'People fully vaccinated per hundred',
    geo = dict(
        showframe = True,
        showlakes = False,
        showcoastlines = True,
        projection = dict(
            type = 'natural earth'
        )
    )
)

fig = dict( data=data, layout=layout )
iplot(fig)

## <a id="3.4">Total vaccinations per months in 2021

In [ ]:
df['date'] = pd.to_datetime(df['date'], errors='coerce')

In [ ]:
plt.figure()
df [df.date.dt.year == 2021].groupby(df.date.dt.month)['total_vaccinations'].sum().reset_index().set_index('date').plot( 
    kind='bar',
    rot=0,
    title =  "Total vaccinations per months in 2021"
    )


In [ ]:
df [df.date.dt.year == 2020].groupby(df.date.dt.month)['total_vaccinations'].sum().reset_index().set_index('date')

Because vaccination started in the world in late 2020, only 2021 is included in the chart above.

# <a id="4"></a><h1 style='background:#26A2AB; border:0; color:black'><center>Prepare Data for Machine learning</center></h1> 

In [ ]:
df.reset_index(drop=True)


## <a id="4.1">Define the desired input and output:

Here we try to predict the value of total_vaccinations by knowing the values of daily_vaccinations and people_fully_vaccinated_per_hundred.

In [ ]:
X = df[['people_fully_vaccinated_per_hundred','daily_vaccinations']]
X

In [ ]:
y = df.total_vaccinations
y

In [ ]:
X_train, X_test, y_train , y_test = train_test_split(X , y , test_size=0.25 , random_state=24)
X_train.shape

## <a id="4.2">Normalize data:

Normalize the data within a particular range. Reason why feature scaling is applied is that few algorithms like gradient descent converge much faster with feature scaling than without it.

<center><img src = "https://i.stack.imgur.com/Yr42l.png" width=50%>

In [ ]:
sc = StandardScaler()
X_train = pd.DataFrame(sc.fit_transform(X_train) , columns=X_train.columns)
X_test  = pd.DataFrame(sc.transform(X_test)     , columns=X_test.columns)

In [ ]:
X_train

# <a id="5"></a><h1 style='background:#26A2AB; border:0; color:black'><center>Train your model</center></h1> 

## <a id="5.1">Random Forest

In [ ]:
rf_regressor = RandomForestRegressor(random_state = 0)

## <a id="5.2">Use GridSearch to select Best Hyperparameters

Exhaustive search over specified parameter values for an estimator

In [ ]:
parameters = {'n_estimators': np.arange(1,11)}
grid_search = GridSearchCV(estimator  = rf_regressor,
                           param_grid = parameters,
                           scoring = 'r2',
                           cv = 10,
                           n_jobs = -1)

grid_search.fit(X_train, y_train)

In [ ]:
best_r2 = grid_search.best_score_
print("Best R^2: {:.2f} %".format(best_r2*100))
best_parameters = grid_search.best_params_
print("Best Parameters:", best_parameters)

## <a id="5.3">Train Model

In [ ]:
%%time
rf_regressor = RandomForestRegressor(n_estimators = 7  , random_state = 0)
rf_regressor.fit(X_train, y_train)

# <a id="6"></a><h1 style='background:#26A2AB; border:0; color:black'><center>Test the model and show the metrics</center></h1> 

In [ ]:
y_predict = rf_regressor.predict(X_test)

In [ ]:
print('R^2 Random Forest model :', r2_score(y_test, y_predict))

# <a id="7"></a><h1 style='background:#26A2AB; border:0; color:black'><center>Save your final model</center></h1> 

In [ ]:
filename = 'RF_model.pkl'
joblib.dump(rf_regressor , filename)